In [1]:
# !pip install seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
# import tensorflow as tf
# import polars as pl
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder, StandardScaler, normalize
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
# import keras
# import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from time import time
import numpy as np
from sklearn.svm import LinearSVC, SVC
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

In [4]:
data = pd.read_csv("Android_Malware.csv")
data.head()

,android.permission.GET_ACCOUNTS,com.sonyericsson.home.permission.BROADCAST_BADGE,android.permission.READ_PROFILE,android.permission.MANAGE_ACCOUNTS,android.permission.WRITE_SYNC_SETTINGS,android.permission.READ_EXTERNAL_STORAGE,android.permission.RECEIVE_SMS,com.android.launcher.permission.READ_SETTINGS,android.permission.WRITE_SETTINGS,com.google.android.providers.gsf.permission.READ_GSERVICES,...,com.android.launcher.permission.UNINSTALL_SHORTCUT,com.sec.android.iap.permission.BILLING,com.htc.launcher.permission.UPDATE_SHORTCUT,com.sec.android.provider.badge.permission.WRITE,android.permission.ACCESS_NETWORK_STATE,com.google.android.finsky.permission.BIND_GET_INSTALL_REFERRER_SERVICE,com.huawei.android.launcher.permission.READ_SETTINGS,android.permission.READ_SMS,android.permission.PROCESS_INCOMING_CALLS,Result
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [5]:
data['Result'].value_counts()

Result
1    14700
0    14632
Name: count, dtype: int64

In [15]:
X = data.drop(columns=['Result'])
y = data['Result']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.90)
X_pca = pca.fit_transform(X_scaled)

X_pca_df = pd.DataFrame(X_pca)
final_df = pd.concat([X_pca_df, y.reset_index(drop=True)], axis=1)

In [16]:
final_df.head()

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,Result
0,-1.083110,-2.884675,-2.837282,0.182551,-1.265597,-2.110831,-1.865732,-3.773236,-2.580564,-1.295771,...,-0.255192,0.787197,-0.584724,-1.668871,-0.363650,0.327369,-0.361544,0.082536,0.128266,0
1,-0.303357,-1.281111,-0.083577,-0.435720,-2.331564,0.187470,-0.810658,1.503930,-0.097968,0.293782,...,-0.423474,-0.488654,-0.033414,0.411934,-1.016776,-0.512490,0.267773,-0.951965,-0.408005,0
2,-0.441660,-1.439841,-0.214177,-0.040709,-2.161533,0.047283,-0.436409,1.109552,-0.204507,0.261527,...,-0.609154,-1.311239,0.252360,0.075188,-0.794364,-0.397431,0.212390,-0.260701,0.044896,0
3,-0.699517,-2.438249,-1.279478,-0.252199,-1.204451,0.095531,-0.100158,-0.015711,0.050757,0.347170,...,-0.228993,-0.430062,0.256856,0.214821,-0.237695,0.218560,0.044156,-0.250332,-0.411699,0
4,-0.379995,-2.017314,-0.964887,-1.262476,-1.457085,-0.457619,-0.846017,1.659861,0.629365,0.823897,...,-1.294758,3.211932,-0.586779,0.331396,0.558879,0.167833,-0.547887,0.931078,-0.148465,0


In [18]:
final_df.shape

(29332, 48)

In [17]:
maxi = 0
for i in range(data.shape[0]):
    if np.sum(data.iloc[i]) > 20:
        maxi += 1
print(maxi)

1386


In [4]:
data.describe()

,android.permission.GET_ACCOUNTS,com.sonyericsson.home.permission.BROADCAST_BADGE,android.permission.READ_PROFILE,android.permission.MANAGE_ACCOUNTS,android.permission.WRITE_SYNC_SETTINGS,android.permission.READ_EXTERNAL_STORAGE,android.permission.RECEIVE_SMS,com.android.launcher.permission.READ_SETTINGS,android.permission.WRITE_SETTINGS,com.google.android.providers.gsf.permission.READ_GSERVICES,...,com.android.launcher.permission.UNINSTALL_SHORTCUT,com.sec.android.iap.permission.BILLING,com.htc.launcher.permission.UPDATE_SHORTCUT,com.sec.android.provider.badge.permission.WRITE,android.permission.ACCESS_NETWORK_STATE,com.google.android.finsky.permission.BIND_GET_INSTALL_REFERRER_SERVICE,com.huawei.android.launcher.permission.READ_SETTINGS,android.permission.READ_SMS,android.permission.PROCESS_INCOMING_CALLS,Result
count,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000,29332.00000,29332.000000,...,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000,29332.000000
mean,0.226033,0.034263,0.045888,0.013603,0.005762,0.171690,0.060071,0.014114,0.11080,0.043741,...,0.029660,0.003887,0.033445,0.034297,0.949032,0.023728,0.023387,0.051718,0.003648,0.501159
std,0.418268,0.181907,0.209247,0.115837,0.075688,0.377117,0.237622,0.117964,0.31389,0.204521,...,0.169652,0.062222,0.179798,0.181994,0.219937,0.152204,0.151133,0.221461,0.060289,0.500007
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
# for i in data.isna().sum():
#     print(i)

In [19]:
x = final_df.drop("Result", axis = 1)
y = final_df['Result']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42, stratify = y)

In [20]:
lgbm = RandomForestClassifier(n_jobs=-1)
param_grid = {
    'n_estimators': [100, 200, 300, 500],  
    'max_features': ['auto', 'sqrt', 'log2'],  
    'max_depth': [None, 10, 20, 30, 40],  
    'bootstrap': [True, False]  
}

grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, scoring='accuracy', n_jobs=-1, verbose=2, cv = 2)
grid_search.fit(x_train, y_train)
print("Best parameters found: ", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
print(classification_report(y_test, y_pred))

Fitting 2 folds for each of 120 candidates, totalling 240 fits
[CV] END bootstrap=True, max_depth=None, max_features=auto, n_estimators=200; total time=   0.0s[CV] END bootstrap=True, max_depth=None, max_features=auto, n_estimators=300; total time=   0.0s

[CV] END bootstrap=True, max_depth=None, max_features=auto, n_estimators=300; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, n_estimators=500; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, n_estimators=500; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, n_estimators=100; total time=   3.5s
[CV] END bootstrap=True, max_depth=None, max_features=log2, n_es

In [21]:
confusion_matrix(y_test, y_pred)

array([[4275,  115],
       [ 176, 4234]])

In [9]:
modelSVC = SVC()
start = time()
modelSVC.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_SVC = modelSVC.predict(x_test)
acc = accuracy_score(pred_SVC , y_test)
cm = confusion_matrix(pred_SVC , y_test)
cr = classification_report(pred_SVC , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 7.242050409317017 seconds
--------------------------------------------------------
0.9648863636363636
--------------------------------------------------------
[[4228  147]
 [ 162 4263]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      4375
           1       0.97      0.96      0.97      4425

    accuracy                           0.96      8800
   macro avg       0.96      0.96      0.96      8800
weighted avg       0.96      0.96      0.96      8800

--------------------------------------------------------


In [10]:
modelLGBM = LGBMClassifier()
start = time()
modelLGBM.fit(x_train , y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_LGBM = modelLGBM.predict(x_test)
acc = accuracy_score(pred_LGBM , y_test)
cm = confusion_matrix(pred_LGBM , y_test)
cr = classification_report(pred_LGBM , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

[LightGBM] [Info] Number of positive: 10290, number of negative: 10242
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003699 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 172
[LightGBM] [Info] Number of data points in the train set: 20532, number of used features: 86
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501169 -> initscore=0.004676
[LightGBM] [Info] Start training from score 0.004676
--------------------------------------------------------
Time taken: 0.3065834045410156 seconds
--------------------------------------------------------
0.9682954545454545
--------------------------------------------------------
[[4262  151]
 [ 128 4259]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4413
           1       0.97

In [11]:
model_RF = RandomForestClassifier(n_jobs=-1)
start = time()
model_RF.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_RF = model_RF.predict(x_test)
acc = accuracy_score(pred_RF , y_test)
cm = confusion_matrix(pred_RF , y_test)
cr = classification_report(pred_RF , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.5523347854614258 seconds
--------------------------------------------------------
0.9702272727272727
--------------------------------------------------------
[[4271  143]
 [ 119 4267]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4414
           1       0.97      0.97      0.97      4386

    accuracy                           0.97      8800
   macro avg       0.97      0.97      0.97      8800
weighted avg       0.97      0.97      0.97      8800

--------------------------------------------------------


In [12]:
model_lr = LogisticRegression(n_jobs = -1, max_iter=1000)
start = time()
model_lr.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_lr = model_lr.predict(x_test)
acc = accuracy_score(pred_lr , y_test)
cm = confusion_matrix(pred_lr , y_test)
cr = classification_report(pred_lr , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 1.2404005527496338 seconds
--------------------------------------------------------
0.9588636363636364
--------------------------------------------------------
[[4183  155]
 [ 207 4255]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      4338
           1       0.96      0.95      0.96      4462

    accuracy                           0.96      8800
   macro avg       0.96      0.96      0.96      8800
weighted avg       0.96      0.96      0.96      8800

--------------------------------------------------------


In [13]:
model_DT = DecisionTreeClassifier()
start = time()
model_DT.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_DT = model_DT.predict(x_test)
acc = accuracy_score(pred_DT , y_test)
cm = confusion_matrix(pred_DT , y_test)
cr = classification_report(pred_DT , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.11562728881835938 seconds
--------------------------------------------------------
0.96375
--------------------------------------------------------
[[4245  174]
 [ 145 4236]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      4419
           1       0.96      0.97      0.96      4381

    accuracy                           0.96      8800
   macro avg       0.96      0.96      0.96      8800
weighted avg       0.96      0.96      0.96      8800

--------------------------------------------------------


In [14]:
model_KN = KNeighborsClassifier()
start = time()
model_KN.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_KN = model_KN.predict(x_test)
acc = accuracy_score(pred_KN , y_test)
cm = confusion_matrix(pred_KN , y_test)
cr = classification_report(pred_KN , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.012224435806274414 seconds
--------------------------------------------------------
0.9638636363636364
--------------------------------------------------------
[[4205  133]
 [ 185 4277]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      4338
           1       0.97      0.96      0.96      4462

    accuracy                           0.96      8800
   macro avg       0.96      0.96      0.96      8800
weighted avg       0.96      0.96      0.96      8800

--------------------------------------------------------


In [15]:
model_NN = MLPClassifier()
start = time()
model_NN.fit(x_train, y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_NN = model_NN.predict(x_test)
acc = accuracy_score(pred_NN , y_test)
cm = confusion_matrix(pred_NN , y_test)
cr = classification_report(pred_NN , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 34.628437995910645 seconds
--------------------------------------------------------
0.9686363636363636
--------------------------------------------------------
[[4269  155]
 [ 121 4255]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      4424
           1       0.96      0.97      0.97      4376

    accuracy                           0.97      8800
   macro avg       0.97      0.97      0.97      8800
weighted avg       0.97      0.97      0.97      8800

--------------------------------------------------------


In [16]:
model_ada = AdaBoostClassifier()
start = time()
model_ada.fit(x_train,y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_ada = model_ada.predict(x_test)
acc = accuracy_score(pred_ada , y_test)
cm = confusion_matrix(pred_ada , y_test)
cr = classification_report(pred_ada , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 1.2184088230133057 seconds
--------------------------------------------------------
0.9578409090909091
--------------------------------------------------------
[[4166  147]
 [ 224 4263]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      4313
           1       0.97      0.95      0.96      4487

    accuracy                           0.96      8800
   macro avg       0.96      0.96      0.96      8800
weighted avg       0.96      0.96      0.96      8800

--------------------------------------------------------


In [17]:
model_gnb = GaussianNB()
start = time()
model_gnb.fit(x_train,y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_gnb = model_gnb.predict(x_test)
acc = accuracy_score(pred_gnb , y_test)
cm = confusion_matrix(pred_gnb , y_test)
cr = classification_report(pred_gnb , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 0.05602097511291504 seconds
--------------------------------------------------------
0.602840909090909
--------------------------------------------------------
[[ 912   17]
 [3478 4393]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.21      0.98      0.34       929
           1       1.00      0.56      0.72      7871

    accuracy                           0.60      8800
   macro avg       0.60      0.77      0.53      8800
weighted avg       0.91      0.60      0.68      8800

--------------------------------------------------------


In [18]:
model_XG = xgb.XGBClassifier()
start = time()
model_XG.fit(x_train,y_train)
end = time()
print("--------------------------------------------------------")
print(f"Time taken: {end - start} seconds")
print("--------------------------------------------------------")
pred_XG = model_XG.predict(x_test)
acc = accuracy_score(pred_XG , y_test)
cm = confusion_matrix(pred_XG , y_test)
cr = classification_report(pred_XG , y_test)
print(acc)
print("--------------------------------------------------------")
print(cm)
print("--------------------------------------------------------")
print(cr)
print("--------------------------------------------------------")

--------------------------------------------------------
Time taken: 1.2242770195007324 seconds
--------------------------------------------------------
0.9686363636363636
--------------------------------------------------------
[[4265  151]
 [ 125 4259]]
--------------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4416
           1       0.97      0.97      0.97      4384

    accuracy                           0.97      8800
   macro avg       0.97      0.97      0.97      8800
weighted avg       0.97      0.97      0.97      8800

--------------------------------------------------------


In [ ]:
def solve(points):
    cnt = 0
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            if points[j] > poinsts[i]:
                cnt += 1
    return cnt